In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Conv2D, Flatten, MaxPool2D, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16


In [3]:
img_height, img_width=(224,224)
batch_size=32

train_data=r"/kaggle/input/flower-recognition-dataset/flowers1/train"
val_data=r"/kaggle/input/flower-recognition-dataset/flowers1/val"
test_data=r"/kaggle/input/flower-recognition-dataset/flowers1/test"

In [20]:
train_datagen = image.ImageDataGenerator(
     preprocessing_function=tf.keras.applications.vgg16.preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical'
)


Found 2588 images belonging to 5 classes.


In [4]:
train_datagen = image.ImageDataGenerator(
     preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size=(img_height, img_width),
    batch_size=16,
    class_mode='categorical'
)


Found 2588 images belonging to 5 classes.


In [21]:
val_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=tf.keras.applications.vgg16.preprocess_input)

validation_set = val_datagen.flow_from_directory(
    val_data,
    target_size=(img_height,img_width),
    batch_size=16,
    class_mode='categorical'
)


Found 860 images belonging to 5 classes.


In [5]:
val_datagen = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input)

validation_set = val_datagen.flow_from_directory(
    val_data,
    target_size=(img_height,img_width),
    batch_size=16,
    class_mode='categorical'
)

Found 860 images belonging to 5 classes.


In [23]:
x, y = next(test_set)
print(x.shape)
print(y.shape)


(16, 224, 224, 3)
(16, 5)


In [24]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Freeze the layers except the last 4 layers
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Create the custom layers atop the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# Connecting base model with new layers
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [27]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint("vgg16_flowers.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='auto')
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=1, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001, verbose=1)

history = model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_set,
    callbacks=[checkpoint, early, reduce_lr]
)


Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step - accuracy: 0.4117 - loss: 1.7790
Epoch 1: val_accuracy improved from -inf to 0.37209, saving model to vgg16_flowers.keras
162/162 ━━━━━━━━━━━━━━━━━━━━ 68s 329ms/step - accuracy: 0.4127 - loss: 1.7750 - val_accuracy: 0.3721 - val_loss: 1.5682 - learning_rate: 1.0000e-04
Epoch 2/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 225ms/step - accuracy: 0.7936 - loss: 0.6157
Epoch 2: val_accuracy improved from 0.37209 to 0.50000, saving model to vgg16_flowers.keras
162/162 ━━━━━━━━━━━━━━━━━━━━ 42s 251ms/step - accuracy: 0.7938 - loss: 0.6153 - val_accuracy: 0.5000 - val_loss: 1.5109 - learning_rate: 1.0000e-04
Epoch 3/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.8476 - loss: 0.4467
Epoch 3: val_accuracy did not improve from 0.50000
162/162 ━━━━━━━━━━━━━━━━━━━━ 41s 243ms/step - accuracy: 0.8476 - loss: 0.4465 - val_accuracy: 0.4686 - val_loss: 1.4868 - learning_rate: 1.0000e-04
Epoch 4/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step - accuracy: 0.859

In [36]:
#test_data_dir = "/path/to/test"  # Set the path to your test directory

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.vgg16.preprocess_input
)

test_generator = test_datagen.flow_from_directory(
    test_data,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False  # Important for correct label ordering in confusion matrix
)

# Evaluate the model on the test data
scores = model.evaluate(test_generator)
print(f"Test Loss: {scores[0]}, Test Accuracy: {scores[1]}")


Found 774 images belonging to 5 classes.


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


25/25 ━━━━━━━━━━━━━━━━━━━━ 27s 503ms/step - accuracy: 0.9118 - loss: 0.5992
Test Loss: 0.5103763937950134, Test Accuracy: 0.9147287011146545


In [37]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Predict the output
test_generator.reset()
pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
predicted_class_indices = np.argmax(pred, axis=1)

# True labels
labels = (test_generator.class_indices)
labels = dict((v, k) for k, v in labels.items())
true_class_indices = test_generator.classes

# Confusion Matrix and Classification Report
conf_matrix = confusion_matrix(true_class_indices, predicted_class_indices)
print("Confusion Matrix:")
print(conf_matrix)

report = classification_report(true_class_indices, predicted_class_indices, target_names=list(labels.values()))
print("Classification Report:")
print(report)


25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 139ms/step
Confusion Matrix:
[[122   4   1   3   4]
 [  2 181   0   4   5]
 [  0   1 124   1  18]
 [  0   4   0 121   4]
 [  1   1  11   2 160]]
Classification Report:
              precision    recall  f1-score   support

       daisy       0.98      0.91      0.94       134
   dandelion       0.95      0.94      0.95       192
        rose       0.91      0.86      0.89       144
   sunflower       0.92      0.94      0.93       129
       tulip       0.84      0.91      0.87       175

    accuracy                           0.91       774
   macro avg       0.92      0.91      0.92       774
weighted avg       0.92      0.91      0.92       774



In [39]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

# Assuming you have a CSV with columns 'filename' and 'label'
labels_df = pd.read_csv('/kaggle/input/flower-test-file/truelabel.csv')

# Directory containing all test images
test_dir = '/kaggle/input/flower-recognition-dataset/flowers1/test_images'

# Prepare the images
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

# Load all images and their labels
test_images = []
labels = []
for _, row in labels_df.iterrows():
    img_path = os.path.join(test_dir, row['image_id'])
    img = load_and_preprocess_image(img_path)
    test_images.append(img)
    labels.append(row['true_label'])

# Convert to numpy arrays
test_images = np.vstack(test_images)  # Shape will be (num_images, 224, 224, 3)
labels = np.array(labels)  # Make sure this is numeric or one-hot encoded as needed


In [41]:
predictions = model.predict(test_images)
predicted_class_indices = np.argmax(predictions, axis=1)

# If labels are not numeric, convert them to numeric using the same mapping used during training
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(labels_df['true_label'])  # Assuming all labels are here
numeric_labels = encoder.transform(labels)

# Metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Accuracy:", accuracy_score(numeric_labels, predicted_class_indices))
print("Confusion Matrix:")
print(confusion_matrix(numeric_labels, predicted_class_indices))
print("Classification Report:")
print(classification_report(numeric_labels, predicted_class_indices, target_names=encoder.classes_))


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
Accuracy: 0.875
Confusion Matrix:
[[17  0  0  1  2]
 [ 0 19  0  0  0]
 [ 0  0 16  0  8]
 [ 0  1  0 18  0]
 [ 0  0  0  1 21]]
Classification Report:
              precision    recall  f1-score   support

       daisy       1.00      0.85      0.92        20
   dandelion       0.95      1.00      0.97        19
        rose       1.00      0.67      0.80        24
   sunflower       0.90      0.95      0.92        19
       tulip       0.68      0.95      0.79        22

    accuracy                           0.88       104
   macro avg       0.91      0.88      0.88       104
weighted avg       0.90      0.88      0.88       104



In [6]:
from tensorflow.keras.regularizers import l2
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

for layer in base_model.layers[:-50]:
    layer.trainable = True


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x) 
x = Dropout(0.5)(x) 
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
from tensorflow.keras.optimizers import Adam

# Instantiate the optimizer with the desired learning rate
optimizer = Adam(learning_rate=0.0001)

# Compile the model with the optimizer
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [8]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight

from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-4)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

steps_per_epoch = max(1, train_generator.samples // batch_size)
validation_steps = max(1, validation_set.samples // batch_size)


model.fit(
    train_generator,
    validation_data=validation_set,
    epochs=30, 
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
     class_weight=class_weights_dict,
    callbacks=[early_stopping, reduce_lr]
)


Epoch 1/30


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1725312016.007450     119 service.cc:145] XLA service 0x7a6440002700 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725312016.007502     119 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1725312016.007506     119 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1725312042.482440     119 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


80/80 ━━━━━━━━━━━━━━━━━━━━ 94s 430ms/step - accuracy: 0.5151 - loss: 1.3330 - val_accuracy: 0.2620 - val_loss: 2.4010 - learning_rate: 1.0000e-04
Epoch 2/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 46s 583ms/step - accuracy: 0.8240 - loss: 0.4878 - val_accuracy: 0.2308 - val_loss: 1.8335 - learning_rate: 1.0000e-04
Epoch 3/30
 2/80 ━━━━━━━━━━━━━━━━━━━━ 11s 146ms/step - accuracy: 0.9062 - loss: 0.2812

/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


80/80 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - accuracy: 0.9367 - loss: 0.2057 - val_accuracy: 0.0714 - val_loss: 2.0439 - learning_rate: 1.0000e-04
Epoch 4/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 24s 282ms/step - accuracy: 0.8681 - loss: 0.3954 - val_accuracy: 0.2596 - val_loss: 3.8208 - learning_rate: 1.0000e-04
Epoch 5/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 20s 254ms/step - accuracy: 0.8704 - loss: 0.3627 - val_accuracy: 0.2452 - val_loss: 2.0707 - learning_rate: 1.0000e-04
Epoch 6/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9691 - loss: 0.1533 - val_accuracy: 0.3214 - val_loss: 1.7342 - learning_rate: 1.0000e-04
Epoch 7/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 22s 253ms/step - accuracy: 0.9136 - loss: 0.2697 - val_accuracy: 0.2572 - val_loss: 2.0800 - learning_rate: 1.0000e-04
Epoch 8/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 19s 244ms/step - accuracy: 0.8803 - loss: 0.3108 - val_accuracy: 0.1971 - val_loss: 1.8582 - learning_rate: 1.0000e-04
Epoch 9/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9051 - los

In [9]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input

# Assuming you have a CSV with columns 'filename' and 'label'
labels_df = pd.read_csv('/kaggle/input/flower-test-file/truelabel.csv')

# Directory containing all test images
test_dir = '/kaggle/input/flower-recognition-dataset/flowers1/test_images'

# Prepare the images
def load_and_preprocess_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

# Load all images and their labels
test_images = []
labels = []
for _, row in labels_df.iterrows():
    img_path = os.path.join(test_dir, row['image_id'])
    img = load_and_preprocess_image(img_path)
    test_images.append(img)
    labels.append(row['true_label'])

# Convert to numpy arrays
test_images = np.vstack(test_images)  # Shape will be (num_images, 224, 224, 3)
labels = np.array(labels)  # Make sure this is numeric or one-hot encoded as needed


In [10]:
predictions = model.predict(test_images)
predicted_class_indices = np.argmax(predictions, axis=1)

# If labels are not numeric, convert them to numeric using the same mapping used during training
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(labels_df['true_label'])  # Assuming all labels are here
numeric_labels = encoder.transform(labels)

# Metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print("Accuracy:", accuracy_score(numeric_labels, predicted_class_indices))
print("Confusion Matrix:")
print(confusion_matrix(numeric_labels, predicted_class_indices))
print("Classification Report:")
print(classification_report(numeric_labels, predicted_class_indices, target_names=encoder.classes_))


4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step
Accuracy: 0.8365384615384616
Confusion Matrix:
[[18  1  1  0  0]
 [ 0 19  0  0  0]
 [ 1  0 21  0  2]
 [ 0  5  2 10  2]
 [ 1  0  2  0 19]]
Classification Report:
              precision    recall  f1-score   support

       daisy       0.90      0.90      0.90        20
   dandelion       0.76      1.00      0.86        19
        rose       0.81      0.88      0.84        24
   sunflower       1.00      0.53      0.69        19
       tulip       0.83      0.86      0.84        22

    accuracy                           0.84       104
   macro avg       0.86      0.83      0.83       104
weighted avg       0.86      0.84      0.83       104

